# Project 3

## Getting Insights into Images and their Metadata


Analysis will include:
...

In [1]:
%matplotlib inline
import scipy
import scipy.io
import numpy as np
import torch
import torchvision
from torchvision import models, transforms
import sqlite3
import matplotlib
from matplotlib import pyplot as plt
from PIL import Image

In [2]:
vgg16 = torchvision.models.vgg16(pretrained=True) # very slow command

C:\Users\saraw\miniconda3\envs\mlfds\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\saraw\miniconda3\envs\mlfds\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


funny, for me that command took maybe half a second? /sara

In [3]:
# vgg16.features is a series of modules.
# we can chain an input through the modules via the forward method.
phi = vgg16.features.forward

def preprocess_image(img):

    # commented out lines are (probably) unnecessary? I hope?
    transform = transforms.Compose([
        #transforms.Resize(256), # or 224
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    batch = transform(img)
    # batch = torch.unsqueeze(batch, 0)
    return batch 

def fetch_img(img_index:int, healthy:bool=True):
    if healthy:
        return Image.open(f"Apple___healthy/image ({img_index}).JPG")
    else:
        return Image.open(f"Apple___Black_rot/image ({img_index}).JPG")

def img_features(img_index:int, healthy:bool=True):
    return phi(preprocess_image(fetch_img(img_index, healthy=healthy)))

phi_out_shape = img_features(1, healthy=True).shape
phi_out_shape


torch.Size([512, 8, 8])

actually I think centercrop and resizing to the size the training data had is relativly common but i don't know if thats neccessary here, I guess it wouldnt work otherwise? /sara <br> anyway, I will also reshape it nevertheless so its 1D and the we'll see /sara

In [17]:
def mean_features(indices, healthy=True):
    mean = None
    for x in indices:
        if mean is None:
            mean = img_features(x, healthy=healthy)
        mean += img_features(x, healthy=healthy)
    return mean

training_indices = range(1, 6) # first few images as training set. Can always be changed later.

mean_healthy = mean_features(training_indices, healthy=True)
mean_rotted = mean_features(training_indices, healthy=False)

w = mean_rotted - mean_healthy
w = torch.flatten(torch.transpose( w / torch.linalg.norm(w), 0, 1 ))

def guess(x):
    return torch.dot(w, torch.flatten(phi(x)))

def img_guess(img_index, healthy=True): # g(x) from the project3 pdf
    return torch.dot(w, torch.flatten(img_features(img_index, healthy)))

# see if we can correctly identify the first healthy/unhealthy leaves not in the training set
test_healthy = img_guess(11, healthy=True)
test_unhealthy = img_guess(11, healthy=False)

test_healthy, test_unhealthy

(tensor(2.4047, grad_fn=<DotBackward0>),
 tensor(1.1867, grad_fn=<DotBackward0>))

interpretation: the first image is "closer" to the mean of helathy images than the second one